In [1]:
import nest_asyncio
nest_asyncio.apply()
del nest_asyncio

In [2]:
import stan

schools_code = """
data {
  int<lower=0> J;         // number of schools
  real y[J];              // estimated treatment effects
  real<lower=0> sigma[J]; // standard error of effect estimates
}
parameters {
  real mu;                // population treatment effect
  real<lower=0> tau;      // standard deviation in treatment effects
  vector[J] eta;          // unscaled deviation from mu by school
}
transformed parameters {
  vector[J] theta = mu + tau * eta;        // school treatment effects
}
model {
  target += normal_lpdf(eta | 0, 1);       // prior log-density
  target += normal_lpdf(y | theta, sigma); // log-likelihood
}
"""

schools_data = {"J": 8,
                "y": [28,  8, -3,  7, -1,  1, 18, 12],
                "sigma": [15, 10, 16, 11,  9, 11, 10, 18]}

posterior = stan.build(schools_code, data=schools_data)

Building: found in cache, done.
Messages from stanc:
Warning in '/var/folders/79/lndg0b5n2q3b40m_ty2kxlm40000gn/T/httpstan_f6j1s3o1/model_qv763guo.stan', line 4, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc
Warning in '/var/folders/79/lndg0b5n2q3b40m_ty2kxlm40000gn/T/httpstan_f6j1s3o1/model_qv763guo.stan', line 5, column 2: Declaration
    of arrays by placing brackets after a variable name is deprecated and
    will be removed in Stan 2.32.0. Instead use the array keyword before the
    type. This can be changed automatically using the auto-format flag to
    stanc


In [3]:
fit = posterior.sample(num_chains=4, num_samples=1000)
eta = fit["eta"]  # array with shape (8, 4000)
df = fit.to_frame()  # pandas `DataFrame, requires pandas
print(df.describe().T)

Sampling:   0%
Sampling:  25% (2000/8000)
Sampling:  50% (4000/8000)
Sampling:  75% (6000/8000)
Sampling: 100% (8000/8000)
Sampling: 100% (8000/8000), done.
Messages received during sampling:
  Gradient evaluation took 6e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.6 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.1 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.3e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.13 seconds.
  Adjust your expectations accordingly!
  Gradient evaluation took 1.2e-05 seconds
  1000 transitions using 10 leapfrog steps per transition would take 0.12 seconds.
  Adjust your expectations accordingly!


                count       mean       std            min        25%  \
parameters                                                             
lp__           4000.0 -39.605211  2.654314  -5.349273e+01 -41.272702   
accept_stat__  4000.0   0.839263  0.224099  9.679479e-166   0.800849   
stepsize__     4000.0   0.379592  0.045424   3.304945e-01   0.339072   
treedepth__    4000.0   3.252250  0.508119   1.000000e+00   3.000000   
n_leapfrog__   4000.0  10.439750  4.155922   1.000000e+00   7.000000   
divergent__    4000.0   0.000250  0.015811   0.000000e+00   0.000000   
energy__       4000.0  44.639227  3.514038   3.520681e+01  42.104190   
mu             4000.0   7.874447  4.898953  -1.224239e+01   4.585197   
tau            4000.0   6.387244  5.362492   5.709867e-03   2.420966   
eta.1          4000.0   0.385289  0.939104  -3.234108e+00  -0.218306   
eta.2          4000.0  -0.006333  0.875417  -3.028142e+00  -0.581792   
eta.3          4000.0  -0.204522  0.949303  -3.352831e+00  -0.85